In [1]:
%%capture
!pip install pandas

In [7]:
import pandas as pd

class Experiment:
    __model_name: str
    __single_call_dataset: pd.DataFrame
    __multi_call_dataset: pd.DataFrame

    def __init__(self, model_name: str, single_call_dataset: pd.DataFrame, multi_call_dataset: pd.DataFrame):
        self.__model_name = model_name
        self.__single_call_dataset = single_call_dataset
        self.__multi_call_dataset = multi_call_dataset
        assert self.__multi_call_dataset is not None
        assert self.__single_call_dataset is not None

    @property
    def model_name(self):
        return self.__model_name

    @property
    def single_call_dataset(self):
        return self.__single_call_dataset.copy()

    @property
    def multi_call_dataset(self):
        return self.__multi_call_dataset.copy()
    
    def __repr__(self):
        return f"{self.__model_name} Experiment"

In [8]:
from typing import List, Dict
from os import listdir
from os.path import join, isdir

resources_path: str = join("..", "resources")

MODELS: List[str] = [d for d in listdir(resources_path) if isdir(join(resources_path, d))]

EXPERIMENTS: Dict[str, Experiment] = {}
for model in MODELS:
    single_call_df_: pd.DataFrame = pd.read_csv(f"{resources_path}/{model}/sampled_reviews_with_output_multicall_{model}.csv")
    multi_call_df_: pd.DataFrame = pd.read_csv(f"{resources_path}/{model}/sampled_reviews_with_output_{model}.csv")
    EXPERIMENTS[model] = Experiment(model_name=model, single_call_dataset=single_call_df_, multi_call_dataset=multi_call_df_)

EXPERIMENTS

{'gemma2_9b': gemma2_9b Experiment,
 'qwen2_7b': qwen2_7b Experiment,
 'llama3.1': llama3.1 Experiment,
 'phi3_medium': phi3_medium Experiment,
 'mistral_7b': mistral_7b Experiment}